### Analyse automatique du narratif 
`Shami THIRION SEN`

# Semantique textuelle



- construction de corpus contrastif
  - corpus communiste vs. «non-communiste»

- défi temps - disponibilité des données 
  - si le tp sem 1 visait  le narratifs des temps communiste, (en hind sight) l'obtention du corpus est un défi non négligeable. À l'époque le Bengal est encore loin de la digitalisation des médias.
  

- 

In [1]:
import numpy as np
import json
import glob
from pprint import pprint
from collections import defaultdict
import os, math
from utils import * # import des fonctions depuis le fichier utils

import gensim
import gensim.corpora as corpora
from gensim.corpora import Dictionary
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

#visualisation
import pyLDAvis
import pyLDAvis.gensim

# outils TAL en bengali
import BnLemma as lm
from bnlp import BengaliPOS

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [2]:
# Bangla POS tagging
bn_pos = BengaliPOS()
from bnlp import BengaliCorpus as corpus
stopwords = corpus.stopwords[:20] + ['রি' + 'টি']

In [3]:
def gen_words(texts):
    single_string = [ ' '.join(line) for line in texts]
    data_words = [gensim.utils.simple_preprocess(text) for text in single_string]
    return data_words


In [4]:
## BIGRAMMES ET TIGRAMMES
def make_bigrams(texts):
    # print(bigram[doc] for doc in texts)
    return ([bigram[doc] for doc in texts])

def make_trigrams(texts):
    return([trigram[bigram[doc]] for doc in texts])


In [5]:
# on choisit le nombre de tokens (pour les 3 corpus)
num_tokens = 50000

#### BanglaGanashakti 
- Lecture du corpus extrait et nettoyage

In [6]:
banglaGanashakti = "../corpus/txtFiles/banglaGanashakti.txt"
filtered_ganashakti = read_corpus(banglaGanashakti, num_tokens)
filtered_ganashakti = [list for list in filtered_ganashakti if len(list)>0] # old list_of_docs


In [7]:
# generation   
data_words  = gen_words(filtered_ganashakti)
print(data_words[9][:200])

# copy_this
bigram_phrases = gensim.models.Phrases(filtered_ganashakti, min_count=5, threshold=50)
trigram_phrases = gensim.models.Phrases(bigram_phrases[filtered_ganashakti], threshold=50)

bigram = gensim.models.phrases.Phraser(bigram_phrases)
trigram = gensim.models.phrases.Phraser(trigram_phrases)

data_bigrams = make_bigrams(filtered_ganashakti)
data_bigrams_trigrams = make_trigrams(data_bigrams)

# data_bigrams_trigrams = make_bigrams_trigrams(filtered_ganashakti)

['আরএসএস', 'অন', 'যবস', 'রহসন']


In [8]:
print(data_bigrams_trigrams[:10])

[['বিধান', 'গণতন্ত্র'], ['বিধান', 'প্রস্তাব', 'তন্ত্র', 'শব্দ', 'সরকার', 'বেসরকারি', 'হাত', 'পরিষ্কার'], ['বিয়োজন', 'চেয়ে', 'বিজেপি', 'সাংসদ', 'সরকার', 'মুখ', 'এমন', 'দেশ'], ['সংসদ', 'বেসরকারি', 'বিলটি', 'সংবাদ', 'ক্ষেত্র', 'দেশ', 'পুঁজি', 'প্রবেশ'], ['মাধ্যম', 'প্রতিবেদন', 'স্বনির্ভরতায়', 'প্রশ্ন', 'চিহ্ন', 'পড়া'], ['লেখা', 'বিধান', 'প্রাস্তাবনায়', 'প্রশ্ন', 'মুখ', 'বিচার', 'সুপ্রিমকোর্ট'], ['নিরপেক্ষ', 'শব্দ', 'সংযোজনা', 'দেশ', 'বিচার', 'যুক্তি', 'প্রমাণ', 'পরিবর্তন'], ['মূর্তি', 'খর', 'মন্তব্য', 'দেশ', 'একাত্তর', 'বিশ্বাস', 'ভিত্তি', 'অযোধ্যা'], ['কাশ্মীর', 'লাদাখ', 'হাইকোর্ট', 'বিচার', 'পঙ্কজ', 'মামলা', 'রায়', 'দান', 'অসুবিধে'], ['থাল', 'আরএসএস', 'অনুষ্ঠান', 'হাজির', 'গোটা', 'ব্যবস্হা', 'প্রহসন']]


In [9]:
### TF-IDF

id2word = Dictionary(data_bigrams_trigrams)
corpus = [id2word.doc2bow(text) for text in filtered_ganashakti]

word = id2word[[9][:1][0]]
print (word)

এমন


In [10]:
corpus=get_corpus(corpus,id2word)
# corpus[:10]
# print(corpus)

In [11]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=20,
                                           random_state=100,
                                           update_every=1,
                                           chunksize=200,
                                           passes=10,
                                           alpha="auto")

In [12]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word, mds="mmds", R=30)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
10     0.173263  0.242754       1        1  8.489961
4      0.077853 -0.329680       2        1  8.354683
14    -0.158582 -0.139313       3        1  6.971033
17    -0.117327  0.236446       4        1  5.983047
19     0.134428 -0.131214       5        1  5.843994
13     0.263688 -0.091672       6        1  5.603805
9      0.020565  0.249792       7        1  5.596261
16    -0.144631 -0.225280       8        1  5.537423
7     -0.230390  0.088346       9        1  5.415043
18     0.197191  0.119668      10        1  5.286448
0      0.193292  0.019428      11        1  5.258481
3      0.001926  0.126361      12        1  4.853890
5     -0.213894 -0.033338      13        1  4.379945
15    -0.103114  0.107994      14        1  3.926385
12     0.010391 -0.160847      15        1  3.517605
6     -0.136598  0.033983      16        1  3.390018
1     -0.025339 -0.095981      17        1  3.326107
11    -0.049127 -0.014480      18        1  2.944147
2      0.017025 -0.006687      19        1  2.747259
8      0.089379  0.003718      20        1  2.574467, topic_info=           Term       Freq      Total Category  logprob  loglift
135       মানুষ  57.000000  57.000000  Default  30.0000  30.0000
268    ভ্যাকসিন  49.000000  49.000000  Default  29.0000  29.0000
7         সরকার  39.000000  39.000000  Default  28.0000  28.0000
11          দেশ  60.000000  60.000000  Default  27.0000  27.0000
296      সংস্থা  27.000000  27.000000  Default  26.0000  26.0000
...         ...        ...        ...      ...      ...      ...
626      পরিমাণ   0.035524   1.060132  Topic20  -7.7974   0.2636
1023     জায়গা   0.035524   1.059965  Topic20  -7.7974   0.2637
1048      বজায়   0.035524   1.060018  Topic20  -7.7974   0.2637
1146  পরিকল্পনা   0.035524   1.060191  Topic20  -7.7974   0.2635
1318     দূরত্ব   0.035524   1.060307  Topic20  -7.7974   0.2634

[743 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
156       1  0.171065      অংশ
156       3  0.342131      অংশ
156       8  0.171065      অংশ
213       7  0.614445   অথরিটি
116       2  0.333647   অধিকার
...     ...       ...      ...
402      15  0.466429  হারিকেন
92        3  0.834942   হিন্দু
175      15  0.719604    হুগলী
352       2  0.629480        ে
297       7  0.614426     ১২টা

[616 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[11, 5, 15, 18, 20, 14, 10, 17, 8, 19, 1, 4, 6, 16, 13, 7, 2, 12, 3, 9])

In [13]:
pyLDAvis.save_html(vis, '../topic_modeling_ganashaki.html')

#### Calcul spécificité

In [14]:
max_specificity, min_specificity = get_higest_lowest_specificity(filtered_ganashakti, 10)

In [15]:
print('Token les plus spécifiques:\n')
pprint(max_specificity)
print('\nToken les moins spécifiques:\n')
pprint(min_specificity)

Token les plus spécifiques:

{'অভিযোগ': 1.7034556832544927,
 'কাগজ': 1.7034556832544927,
 'ঘটনা': 1.7034556832544927,
 'দেশ': 2.822366552123779,
 'বছর': 3.981652821276082,
 'বিরোধ': 1.7034556832544927,
 'মুখ': 1.3058448579813575,
 'রাজ্য': 1.7034556832544927,
 'সংস্থা': 2.4176201267623734,
 'সভা': 1.4214206099086821}

Token les moins spécifiques:

{'আবেদন': -1.4734260601920461,
 'তৈরি': -0.6995223437085923,
 'প্রশাসক': -1.1313767185499328,
 'ভারত': -0.5644679352351545,
 'ভ্যাকসিন': -1.1313767185499328,
 'মঙ্গলবার': -0.9112545540878407,
 'মন': -0.5976209869719409,
 'মানুষ': -1.846381145742381,
 'সরকার': -0.7250544362478527,
 'হাইকোর্ট': -0.6995223437085923}


### Bartamanpatrika

In [16]:
bartamanpatrika = "../corpus/txtFiles/bartamanpatrika.txt"
filtered_bartamanpatrika = read_corpus(bartamanpatrika, num_tokens)
filtered_bartamanpatrika = [list for list in filtered_bartamanpatrika if len(list)>0]

In [17]:
# generation   
data_words  = gen_words(filtered_bartamanpatrika)
# print(data_words[9][:200])

# copy_this
bigram_phrases = gensim.models.Phrases(filtered_bartamanpatrika, min_count=5, threshold=50)
trigram_phrases = gensim.models.Phrases(bigram_phrases[filtered_bartamanpatrika], threshold=50)

bigram = gensim.models.phrases.Phraser(bigram_phrases)
trigram = gensim.models.phrases.Phraser(trigram_phrases)

data_bigrams = make_bigrams(filtered_bartamanpatrika)
data_bigrams_trigrams = make_trigrams(data_bigrams)


In [18]:
print(data_bigrams_trigrams[:10])

[['শত', 'নেতৃত্ব', 'ক্রিকেট', 'কন্ট্রোল', 'বোর্ড', 'প্রস্তাব', 'গ্রহণ', 'শত', 'টেস্ট', 'অধিনায়ক', 'ম্যাচ', 'মাইলস্টোন', 'ম্যাচ', 'চাওয়া', 'অভিমানী', 'কে'], ['টেস্ট', 'অধিনায়ক', 'কোকিল', 'ইস্তফা', 'পাঁচ', 'বুমরাহকেও'], ['হ্যামস্ট্রিংয়ে', 'চোট', 'ওয়েটিং', 'ইন্ডিজের', 'বিরুদ্ধ', 'সিরিজ', 'শর্করা', 'ফেব্রুয়ারি', 'গোড়ায়', 'ভারত'], ['অধিনায়ক', 'অধীনে', 'কাজ'], ['ের', 'ব্যালট', 'ডিও', 'সি', 'ল্যাং', 'বা', 'পায়ে', 'জাদু', 'হার'], ['ভ্যাকসিন', 'নেব', 'নোভাক', 'জকোভিচ', 'প্রশ্ন', 'বিশ্ব'], ['ভার', 'গ্যালারি', 'নোনোভাক', 'লেখা', 'পোস্ট', 'জকোভিচ', 'অস্ট্রেলিয়ান'], ['প্রত্যাখ্যাত', 'বিলাত', 'গোলে', 'সুপার', 'কাপ', 'ঘর', 'রিয়াল', 'মাদ্রাসা', 'রবিবার', 'সৌদি', 'আরব'], ['আইএসএলে', 'রানার', 'মোহন', 'বাগান', 'সুবাদ', 'ব্রিগেড', 'কাপের', 'প্লেঅফে'], ['আইএসএলে', 'সোমবার', 'জাম', 'হায়', 'এফ', 'ম্যাচ']]


In [19]:
### TF-IDF

id2word = Dictionary(data_bigrams_trigrams)
corpus = [id2word.doc2bow(text) for text in filtered_bartamanpatrika]

word = id2word[[9][:1][0]]
print (word)

প্রস্তাব


In [20]:
corpus=get_corpus(corpus,id2word)


In [21]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=30,
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha="auto")



In [22]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word, mds="mmds", R=30)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
5     -0.255614 -0.283250       1        1  9.032091
6      0.061873 -0.385425       2        1  8.845939
1     -0.361885 -0.011121       3        1  6.664382
28     0.317061 -0.025626       4        1  6.499445
4      0.272054 -0.203203       5        1  6.163228
24     0.246044  0.104077       6        1  4.521679
16     0.190597  0.226003       7        1  4.414358
19     0.034703 -0.289128       8        1  4.112245
9     -0.216219 -0.063405       9        1  3.977624
2     -0.113885 -0.195093      10        1  3.961161
26    -0.231323  0.126767      11        1  3.673707
23    -0.145200  0.150540      12        1  3.015557
14     0.002165 -0.109628      13        1  2.862973
0     -0.084288 -0.086434      14        1  2.853757
10     0.041619  0.237002      15        1  2.812068
3     -0.036990  0.217879      16        1  2.766005
29    -0.088966  0.180779      17        1  2.500013
12    -0.133885  0.016050      18        1  2.469168
11     0.073263 -0.122695      19        1  2.443823
27     0.153031 -0.064045      20        1  2.438160
7      0.137924  0.027128      21        1  2.236164
21     0.081943  0.147543      22        1  1.966258
25    -0.068549  0.046271      23        1  1.945932
8      0.072969  0.090401      24        1  1.867854
18    -0.024478  0.064687      25        1  1.483906
15     0.000352  0.109282      26        1  1.296101
13     0.038137 -0.002257      27        1  1.191757
22     0.034325  0.038113      28        1  1.185352
20     0.000735  0.029321      29        1  0.446143
17     0.002486  0.029469      30        1  0.353148, topic_info=               Term       Freq      Total Category  logprob  loglift
97            বৃদ্ধ  76.000000  76.000000  Default  30.0000  30.0000
105          সাফল্য  53.000000  53.000000  Default  29.0000  29.0000
98          ব্যবসায়  50.000000  50.000000  Default  28.0000  28.0000
232           সরকার  25.000000  25.000000  Default  27.0000  27.0000
81              বছর  28.000000  28.000000  Default  26.0000  26.0000
..              ...        ...        ...      ...      ...      ...
25           শর্করা   0.008537   1.568579  Topic30  -7.4236   0.4325
26            সিরিজ   0.008537   4.403272  Topic30  -7.4236  -0.5997
27   হ্যামস্ট্রিংয়ে   0.008537   1.568579  Topic30  -7.4236   0.4325
28            অধীনে   0.008537   2.762247  Topic30  -7.4236  -0.1334
29              কাজ   0.008537  12.310416  Topic30  -7.4236  -1.6278

[1242 rows x 6 columns], token_table=      Topic      Freq            Term
term                                 
251       3  0.894978             অংশ
233      12  0.626175       অক্সফোর্ড
93        2  0.745549          অগ্রসর
93        8  0.237220          অগ্রসর
85       17  0.658145             অঙ্
...     ...       ...             ...
357      17  0.409202             হোম
27       16  0.637520  হ্যামস্ট্রিংয়ে
329      15  0.451920             াদি
38       28  0.407902              ের
148      18  0.410273              য়ে

[512 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[6, 7, 2, 29, 5, 25, 17, 20, 10, 3, 27, 24, 15, 1, 11, 4, 30, 13, 12, 28, 8, 22, 26, 9, 19, 16, 14, 23, 21, 18])

In [23]:
pyLDAvis.save_html(vis, '../topic_modeling_bartamanpatrika.html')

#### Calcul de spécificité

In [24]:
max_specificity, min_specificity = get_higest_lowest_specificity(filtered_bartamanpatrika, 10)
# max_specificity, min_specificity

In [25]:
print('Token les plus spécifiquess:\n')
pprint(max_specificity)
print('\nToken les moins spécifiques:\n')
pprint(min_specificity)

Token les plus spécifiquess:

{'অগ্রসর': 2.213898518699129,
 'অর্থকড়ি': 1.1917822053964344,
 'আনন্দ': 1.194358070023163,
 'উপার্জন': 2.4791153722921853,
 'কাজ': 2.391591541360233,
 'পদ': 1.4933962140349704,
 'বিদ্যা': 2.175673473348266,
 'ব্যবসায়': 4.993389359623521,
 'যোগ': 2.640928523468916,
 'সুখ': 1.194358070023163}

Token les moins spécifiques:

{'কর্মী': -1.2203845358262584,
 'কলকাতা': -2.725720195793848,
 'গৃহে': -1.473846371473666,
 'টাকা': -1.282643850251739,
 'প্রেম': -2.225818409875582,
 'বুধবার': -2.263926350293847,
 'মমতা': -1.2203845358262584,
 'মানুষ': -1.473846371473666,
 'রাজ্জাক': -2.5345546860291215,
 'রাজ্য': -2.263926350293847}


### Anandabazar

- corpus qui avec 1000 tokens pointe vers cricket et le sujet de sport 
- Alors qu'avec 10000 nous obtenons plus de généralité

In [26]:
anandabazar = "../corpus/txtFiles/anandabazar.txt"
filtered_anandabazar = read_corpus(anandabazar, num_tokens)
filtered_anandabazar = [list for list in filtered_anandabazar if len(list)>0] 

In [27]:
# generation   
data_words  = gen_words(filtered_anandabazar)
# print(data_words[9][:200])

# copy_this
bigram_phrases = gensim.models.Phrases(filtered_anandabazar, min_count=5, threshold=50)
trigram_phrases = gensim.models.Phrases(bigram_phrases[filtered_anandabazar], threshold=50)

bigram = gensim.models.phrases.Phraser(bigram_phrases)
trigram = gensim.models.phrases.Phraser(trigram_phrases)

data_bigrams = make_bigrams(filtered_anandabazar)
data_bigrams_trigrams = make_trigrams(data_bigrams)


In [28]:
print(data_bigrams_trigrams[:10])

[['১৫টা', 'লোক', 'ঘিরে'], ['ষষ্ঠী', 'সপ্তম', 'তুলনা', 'রবিবার', 'গা', 'রাস্তা', 'গা', 'উত্তর', 'দক্ষিণ', 'নিয়ন্ত্রণ', 'বড়', 'পুজোগুলোয়', 'চোখ', 'পড়া'], ['রাজ্য', 'বিজড়িত', 'ভাঙা', 'দশ', 'রাম', 'সুনীল', 'বনসাই', 'দল', 'নেতৃত্ব', 'পর্যবেক্ষণ', 'মঙ্গল', 'পরিকল্পনা', 'পালন', 'দায়িত্ব'], ['কোটি', 'গুটখায়', 'রঙ', 'শহর'], ['শীর্ষ', 'আদালত', 'মানুষ', 'আস্থা', 'বজায়', 'দায়িত্ব', 'কার'], ['সাল', 'বিধান', 'নির্বচন', 'শুরু', 'বিজড়িত', 'সাল', 'বিতর্ক', 'আরাধন', 'সুকান্ত', 'বারেক', 'পুজোতেই', 'সুলতান'], ['কংগ্রেস', 'প্রস্তাব', 'বাদ', 'শুরু', 'জোট'], ['ডিএ', 'মামলা', 'ধাক্কা', 'খেল', 'রাজ্য', 'সরকার'], ['সোমবার', 'রাত', '৮টা', 'মৃত্যু', 'মৃত', 'পরিবার', 'দেহ', 'নিতে', 'স্বীকার'], ['বারেক', 'চেষ্টায়', 'দিন', 'যোগ', 'সরকার', 'হা', 'অনুমোদন', 'রাহুলপ্রিয়ঙ্কা']]


In [29]:
### TF-IDF

id2word = Dictionary(data_bigrams_trigrams)
corpus = [id2word.doc2bow(text) for text in filtered_anandabazar]

word = id2word[[9][:1][0]]
print (word)

corpus=get_corpus(corpus,id2word)
# corpus[:10]

পড়া


In [30]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=20,
                                           random_state=100,
                                           update_every=1,
                                           chunksize=200,
                                           passes=10,
                                           alpha="auto")



In [31]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word, mds="mmds", R=30)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4     -0.293253 -0.154267       1        1  10.417991
5     -0.326098  0.082835       2        1   9.064946
3      0.031513 -0.283560       3        1   6.236905
6     -0.099848 -0.254080       4        1   5.675408
8      0.084120  0.278498       5        1   5.645226
19    -0.120949  0.260799       6        1   5.462589
16    -0.199943  0.125003       7        1   5.406919
10    -0.139923 -0.147872       8        1   5.191573
17     0.126363 -0.216142       9        1   5.172704
14     0.237692 -0.148764      10        1   5.062809
12     0.274742 -0.030141      11        1   4.520269
13     0.234394  0.139117      12        1   4.375810
2     -0.131655 -0.026376      13        1   4.328196
11    -0.083801  0.095460      14        1   4.322663
0     -0.007014  0.199962      15        1   4.121949
9      0.107769  0.143756      16        1   3.550202
1      0.004065 -0.090711      17        1   3.154141
18     0.130099 -0.033588      18        1   3.102088
15     0.144353  0.055191      19        1   3.020162
7      0.027371  0.004881      20        1   2.167451, topic_info=         Term       Freq      Total Category  logprob  loglift
171       দেশ  27.000000  27.000000  Default  30.0000  30.0000
157   রাজ্জাক  34.000000  34.000000  Default  29.0000  29.0000
223       বছর  26.000000  26.000000  Default  28.0000  28.0000
58      সরকার  26.000000  26.000000  Default  27.0000  27.0000
335      ভারত  20.000000  20.000000  Default  26.0000  26.0000
...       ...        ...        ...      ...      ...      ...
666    প্রচার   0.497577   8.224888  Topic20  -5.1323   1.0264
1487   পাহারা   0.031804   0.919329  Topic20  -7.8824   0.4676
1406    কালীর   0.031800   0.919406  Topic20  -7.8826   0.4674
489      দফতর   0.033188  17.321153  Topic20  -7.8398  -2.4259
335      ভারত   0.031963  20.478016  Topic20  -7.8775  -2.6309

[751 rows x 6 columns], token_table=      Topic      Freq    Term
term                         
722       9  0.648465     অংশ
705      10  0.667768   অগস্ট
598       4  0.729707   অঞ্চল
228      11  0.843509  অধিকার
466       8  0.486742  অনলাইন
...     ...       ...     ...
249       3  0.789427     য়াদ
126      19  0.593163      য়ে
100      11  0.664877    য়েরা
2         8  0.647288    ১৫টা
67       19  0.471756     ৮টা

[760 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[5, 6, 4, 7, 9, 20, 17, 11, 18, 15, 13, 14, 3, 12, 1, 10, 2, 19, 16, 8])

In [32]:
pyLDAvis.save_html(vis, '../topic_modeling_anandabazar.html')

#### Calcul de spécificité

In [33]:
max_specificity, min_specificity = get_higest_lowest_specificity(filtered_anandabazar, 10)

In [34]:
print('Token les plus spécifiques:\n')
pprint(max_specificity)
print('\nToken les moins spécifiques:\n')
pprint(min_specificity)

Token les plus spécifiques:

{'কলকাতা': 1.5778801614714248,
 'গা': 1.4970557220813931,
 'দিন': 1.4548077207324708,
 'দিল্লি': 1.4970557220813931,
 'নেতা': 1.5965450463804507,
 'পশ্চিম': 1.4970557220813931,
 'প্রাক্তন': 1.4970557220813931,
 'ভোট': 3.258295599096833,
 'ম্যাচ': 1.4970557220813931,
 'হাসপাতাল': 1.8719243107077876}

Token les moins spécifiques:

{'ঘণ্টা': -1.9391756661987243,
 'ছবি': -1.211895433809877,
 'টাকা': -1.3875543891087163,
 'দেশ': -1.1934798488373901,
 'ধার': -1.2700287066563456,
 'পাকিস্তান': -1.074699978134664,
 'বিজেপি': -1.5146775696522612,
 'বৃহস্পতিবার': -1.2700287066563456,
 'সোমবার': -1.2700287066563456,
 'হাত': -1.4695469731929143}
